In [ ]:
# ===============================================
#     Preamble
# ===============================================

using DynamicProgramming, JSON
include("ecow.jl")

using Plots
const mm = Plots.mm
fntsm = Plots.font("times", 12)
fntlg = Plots.font("times", 14)
default(
    titlefont=fntlg,
    guidefont=fntlg,
    tickfont=fntsm,
    legendfont=fntsm,
    left_margin=3mm,
    bottom_margin=10mm,
    top_margin=0mm,
    right_margin=0mm,
    size=(500,300),
    xlabel="Days of the Season\n",
    label="",
    w=3,
    c="#00467F"
)
gr()

In [ ]:
# ===============================================
#     Plots for the section: Intuition
# ===============================================

f(q) = min(2.0, q/7.5)
g(ρ, Q) = ρ * f(Q/ρ - 2.75)

Ρ = 0:0.1:3
Q = 0:1:60
M = [
    max(0.0, g(ρ, q)) for ρ in Ρ, q in Q
]
contour(Q, Ρ, M,
    levels = 20,
    title  = "Milk Produced (kg/ha/day)",
    xlabel = "Q (kg/ha/day)\n",
    ylabel = "Stocking Rate (cows/ha)"
)
savefig("moo_intuition.pdf")

In [ ]:
# ===============================================
#     Plots for the section: Example Simulation
# ===============================================

cow = ECow.Cow()
herbage = ECow.FoodOffer(ECow.Pasture(), 35)
supplement = ECow.FoodOffer(ECow.PKE(), 2)

milkingplan(t,n=365) = vcat(trues(t), falses(n-t))

results = Dict(
    :bcs         => vcat(3.2, zeros(365)),
    :maintenance => vcat(63, zeros(365)),
    :lactation   => milkingplan(38 * 7),  # milk for 40 weeks
    :supplement  => zeros(365),
    :herbage     => zeros(365),
    :milksolids  => zeros(365)
)

for day in 1:365
    (new_bcs, new_maintenance, actual_herbage_intake, actual_supplement_intake, milksolids) = ECow.updateday!(
        cow, day, results[:bcs][day], results[:maintenance][day], results[:lactation][day], supplement, herbage
    )
    results[:bcs][day+1]         = new_bcs
    results[:maintenance][day+1] = new_maintenance
    results[:supplement][day]    = actual_supplement_intake
    results[:herbage][day]       = actual_herbage_intake
    results[:milksolids][day]    = milksolids
end

bcs_plot = plot(results[:bcs],
    ylabel="BCS\n(Units)",
    title="(a)",
    ylims=(0,5),
    left_margin=10mm
)

mai_plot = plot(results[:maintenance],
    ylabel="Maintenance\n(MJ/day)",
    title="(b)",
    ylims=(50,80),
    left_margin=10mm
)

her_plot = plot(
    ylabel="Feed Intake\n(kg/day)",
    title="(c)",
    ylims=(0,12.5),
    legend=:topleft,
    left_margin=10mm
)
plot!(results[:herbage], label="Herbage")
plot!(results[:supplement], label="Palm Kernel", linestyle=:dash)

mlk_plot = plot(results[:milksolids],
    ylabel="Milk Solids\n(kg/day)",
    title="(d)",
    ylims=(0,2.5),
    left_margin=10mm
)

plot(bcs_plot, mai_plot, her_plot, mlk_plot, size=(1000, 600))

savefig("moo_example.pdf")

In [ ]:
#
#    Estimate milk correction factor of 1.23
#
# function simulateseason(pasture, palm_kernel, lactation_length)
#     cow = ECow.Cow()
#     herbage = ECow.FoodOffer(ECow.Pasture(), pasture)
#     supplement = ECow.FoodOffer(ECow.PKE(), palm_kernel)

#     milkingplan(t,n=365) = vcat(trues(t), falses(n-t))

#     results = Dict(
#         :bcs         => vcat(3.2, zeros(365)),
#         :maintenance => vcat(63, zeros(365)),
#         :lactation   => milkingplan(lactation_length),  # milk for 40 weeks
#         :supplement  => zeros(365),
#         :herbage     => zeros(365),
#         :milksolids  => zeros(365)
#     )

#     for day in 1:365
#         (new_bcs, new_maintenance, actual_herbage_intake, actual_supplement_intake, milksolids) = ECow.updateday!(
#             cow, day, results[:bcs][day], results[:maintenance][day], results[:lactation][day], supplement, herbage
#         )
#         results[:bcs][day+1]         = new_bcs
#         results[:maintenance][day+1] = new_maintenance
#         results[:supplement][day]    = actual_supplement_intake
#         results[:herbage][day]       = actual_herbage_intake
#         results[:milksolids][day]    = milksolids
#     end

#     sum(results[:herbage]) * 3, sum(results[:supplement]) * 3, sum(results[:milksolids]) * 3
# end

# # Season: 2013/14
# #     herbage:       12.6 t/ha
# #     supplement:     2.8 t/ha
# #     milk solids: 1240.0 kg/ha
# h, s, ms = simulateseason(37.6135, 2.5571, 275)
# @show h, s, ms
# @show h / 12600, s / 2800, ms / 1240

# # Season: 2014/15
# #     herbage:       11.7 t/ha
# #     supplement:     2.9 t/ha
# #     milk solids: 1146.0 kg/ha
# h, s, ms = simulateseason(33.5115, 2.64844, 256)
# @show h, s, ms
# @show h / 11700, s / 2900, ms / 1146

In [ ]:
function plotsummary(resultfile, paramfile, stockingrate=params["stocking_rate"])
    summary = JSON.parsefile(resultfile)
    params = JSON.parsefile(paramfile)

    # ==========================================
    # back out the pasture cover
    cover = zeros(length(summary["herbage"]))
    i = 1
    current_cover = 2500.0
    for week in 1:52
        for day in 1:7
            current_cover += params["grass_growth"][week] -
                stockingrate * summary["herbage"][i]
            cover[i] = current_cover
            i += 1
        end
    end
    # ==========================================
    plot(
        plot(summary["bcs"],
            title="(a)", ylabel="BCS\n(Units)",
            ylims=(0, 5),
            left_margin=10mm
        ),
        plot(summary["maintenance"],
            title="(b)", ylabel="Maintenance\n(MJ/day)",
            ylims=(50, 80),
            left_margin=10mm
        ),
        plot(summary["milksolids"],
            title="(c)", ylabel="Milk Solids\n(kg/cow/day)",
            ylims=(0, 2.5),
            left_margin=10mm
        ),
        plot(cover,
            title="(d)", ylabel="Pasture Cover\n(kg/ha)",
            ylims=(0, 3500),
            left_margin=10mm
        ),
        plot(summary["herbage"],
            title="(e)", ylabel="Herbage\n(kg/cow/day)",
            ylims=(0, 17.5),
            left_margin=10mm
        ),
        plot(summary["supplement"],
            title="(f)", ylabel="Palm Kernel\n(kg/cow/day)",
            ylims=(0, 6),
            left_margin=10mm
        ),
        size=(1500,600)
    )
end

In [ ]:
function summarize(result_file, stocking_rate = 3, milk_price = 6)
   summary = JSON.parsefile(result_file)
   println("Lactation Length:  ",  sum(summary["lactation"]) / 7)
    println("Milksolids kg/Ha:  ",  round(Int, summary["total_milksolids"] * stocking_rate))
    println("Milksolids kg/Cow: ",  round(Int, summary["total_milksolids"]))
    milk_profit = summary["total_milksolids"] * milk_price * stocking_rate
    println("Milksolids \$/Ha:   ", round(Int, milk_profit))
    println("Pasture kg/Ha:     ",  round(0.001 * summary["total_herbage"] * stocking_rate, 2))
    println("Palm Kernel kg/Ha: ",  round(0.001 * summary["total_supplement"] * stocking_rate, 2))
    println("Feed Imported %:   ",  round(100 * summary["total_supplement"] / (summary["total_herbage"] + summary["total_supplement"]), 1))
    palm_expenses = summary["total_supplement"] * 0.5 * stocking_rate
    println("Palm Kernel \$/Ha:  ", round(Int, palm_expenses))
    println("Operating Profit \$/Ha:  ", round(Int, milk_profit - palm_expenses - 3536))
    println("FEI Penalty \$/Ha:  ", round(Int, sum(0.25(summary["supplement"] - 3)) * stocking_rate))
end
# ===============================================
#     Plots for the section: Case Study
# ===============================================

PARAMS, RESULTS, SOLVED_MODEL = "case_study/moo.params", "case_study/moo.results", "case_study/moo.solved.model"
plotsummary(RESULTS, PARAMS)
savefig("moo_simulation.pdf")

summary = JSON.parsefile(RESULTS)
params = JSON.parsefile(PARAMS)
# summarize 
summarize(RESULTS)
#=
    Plot the value to go.
=#
m = open(SOLVED_MODEL, "r") do io
   deserialize(io) 
end

function plotvaluetogo(stage, title="")
    X = 2.5:0.1:4.5
    Y = 2000:50:3000
    stage_one_surface = [
        -m.stages[stage].interpolatedsurface[bcs,70.0,1.0,cover]
        for bcs in X, cover in Y
    ]
    contour(X, Y, stage_one_surface',
        title        = title,
        xlabel       = "BCS\n",
        ylabel       = "Pasture Cover (kg/ha)",
        right_margin = 7.5mm,
        left_margin  = 5mm, levels=30, w=1
    )
end
plot(
    plotvaluetogo(1, "(a) Week 1"),
    plotvaluetogo(25, "(b) Week 25"),
    plotvaluetogo(50, "(c) Week 50"),
    size=(1500, 300), layout=(1,3)
)
savefig("moo_value-to-go.pdf")

In [ ]:
# ===============================================
#     Plots for the section: Two Stage Model.
# ===============================================
function getM(Q,Ρ)
    Qi = 0:0.02:1
    M2  = zeros(length(Ρ), length(Q))

    for (j, q) in enumerate(Q)
        for (i, p) in enumerate(Ρ)
            qx = abs.(q - Qi * p * 6 * 7 * 52)
            qq = Qi[indmin(qx)]
            d = JSON.parsefile("two_stage_results/$(p)_$(qq)_moo.results")
            M2[i,j] = p * sum(d["milksolids"])
        end
    end

    M = copy(M2)
    λ = 1.0
    for col in 1:(size(M, 2) - 0)
        for row in 1:(size(M, 1) - 0)
            M[row, col] = (1-λ) * M2[row, col] + λ * mean(M2[max(1,row-1):min(size(M2, 1), row+1), max(1,col-1):min(size(M2, 2), col+1)])
        end
    end
    M
end

Q  = 0:100:10_000
Ρ  = 2.0:0.05:4.0

M = getM(Q, Ρ)

contour(Q, Ρ, M,
    xlabel = "Total Quantity of Palm Kernel (kg/ha)\n",
    ylabel = "Stocking Rate (cows/ha)",
    right_margin = 5mm, levels=30, w=1
)
savefig("moo_total_milksolids.pdf")

Prices = [6.1,7.6,6.08,5.84,8.4,4.4,3.9, 6.12]
Π = zeros(length(Prices), size(M, 1), size(M, 2))

for (k,q) in enumerate(Q)
    for (j,ρ) in enumerate(Ρ)    
        for (i,price) in enumerate(Prices)
            Π[i,j,k] = price * M[j,k] - (1467 + 589ρ) - 0.5*q
        end
    end
end

# ===============================================
#     Wait and See
# ===============================================
wait_and_see = [
    maximum(Π[i, :, :])
    for (i,price) in enumerate(Prices)
]

xρ = Float64[]
xq = Float64[]
for price in Prices
    idx = findfirst(Prices, price)
    (ρi, qi) = ind2sub(Π[idx, :, :], indmax(Π[idx, :, :]))
    push!(xρ, Ρ[ρi])
    push!(xq, Q[qi])
    if price in [3.9, 6.08, 8.4]
        qx = abs.(Q[qi] - Qi * Ρ[ρi] * 6 * 7 * 52)
        qq = Qi[indmin(qx)]
        plotsummary("two_stage_results/$(Ρ[ρi])_$(qq)_moo.results", "two_stage_results/moo.params", Ρ[ρi])
        savefig("moo_$(round(Int, 10*price))_simulation.pdf")
        summarize("two_stage_results/$(Ρ[ρi])_$(qq)_moo.results", Ρ[ρi], price)
    end
end

println("Wait-and-See: ", round(mean(wait_and_see), 2),"  ", round(std(wait_and_see), 2))
function latexprint(x::Vector)
    print(x[1])
    for xi in x[2:end]
        print(" & ", xi)
    end
    println()
end
latexprint(round.(xρ, 2))
latexprint(round.(xq / 1000, 1))
latexprint(round.(Int, wait_and_see))


# ===============================================
#     Here and Now
# ===============================================
expected_profit = [
    mean(Π[:,j,k]) for j in 1:length(Ρ), k in 1:length(Q)
]
(ρi, qi) = ind2sub(expected_profit, indmax(expected_profit))
qx = abs.(Q[qi] - Qi * Ρ[ρi] * 6 * 7 * 52)
qq = Qi[indmin(qx)]
plotsummary("two_stage_results/$(Ρ[ρi])_$(qq)_moo.results", "two_stage_results/moo.params", Ρ[ρi])
savefig("moo_hereandnow_simulation.pdf")

here_and_now = Π[:, ρi, qi]
println("Here-and-Now: ", round(mean(here_and_now), 2),"  ", round(std(here_and_now), 2))
latexprint(fill(round.(Ρ[ρi], 2), 8))
latexprint(fill(round(Q[qi] / 1000, 1), 8))
latexprint(round.(Int, here_and_now))

# ===============================================
#     EVPI
# ===============================================
println("EVPI = ", mean(wait_and_see) - mean(here_and_now))

# ===============================================
#     Risk
# ===============================================
utility(x, alpha=1) = (1 - exp(-alpha/ 1000 * x )) / (alpha / 1000)

function computeriskaversesolution(alpha, Π, Ρ, Q)
    expected_profit_risk = [
        mean(utility.(Π[:,j,k], alpha)) for j in 1:length(Ρ), k in 1:length(Q)
    ]
    (ρi, qi) = ind2sub(expected_profit_risk, indmax(expected_profit_risk))
    here_and_now_risk = Π[:, ρi, qi]
    Ρ[ρi], Q[qi], mean(here_and_now_risk), std(here_and_now_risk), minimum(here_and_now_risk)
end

expected_profit = [
    mean(Π[:,j,k]) for j in 1:length(Ρ), k in 1:length(Q)
]
std_profit = [
    std(Π[:,j,k]) for j in 1:length(Ρ), k in 1:length(Q)
]

mean_plot = contour(Q, Ρ, expected_profit,
    title  = "(a) Expectation",
    xlabel = "\$\\mathbf{x_1^q}\$",
    ylabel = "\$;;;\\mathbf{x_1^\\rho}\$",
    right_margin=5mm, bottom_margin=5mm,
    ylims=(2.0, 4.0), xlims=(0,10_000), levels=30, w=1
)
std_plot = contour(Q, Ρ, std_profit,
    title  = "(b) Standard Deviation",
    xlabel = "\$\\mathbf{x_1^q}\$",
    ylabel = "\$\\mathbf{x_1^\\rho}\$",
    right_margin=5mm, left_margin=0mm, bottom_margin=5mm,
    ylims=(2.0, 4.0), xlims=(0,10_000), levels=30, w=1
)

for α in [0.1, 1, 10]
    p, q, μ, σ, mn = computeriskaversesolution(α, Π, Ρ, Q)
    @show p, q, μ, σ, mn
    println("α = $(α)), μ=$(round(μ, 2)), σ=$(round(σ, 2)), worst=$(round(mn, 2))")
    scatter!(mean_plot, [q], [p+0.01], c="black", w=0)
    scatter!(std_plot, [q], [p+0.01], c="black", w=0)
    annotate!(mean_plot, [(q+1750,p+0.03,"\$\\alpha=$(α)\$")])
    annotate!(std_plot, [(q+1750,p+0.03,"\$\\alpha=$(α)\$")])
end

scatter!(mean_plot, [2850], [3], c="red", w=0)
scatter!(std_plot, [2850], [3], c="red", w=0)
annotate!(mean_plot, [(2850+1750,3+0.03,"case farm")])
annotate!(std_plot, [(2850+1750,3+0.03,"case farm")])
    

plot(mean_plot, std_plot, size=(1000, 300))
savefig("moo_risk.pdf")